# Neurális háló tanítása

#### A hiperparamétereket az optim.ipynb-ben választjuk ki, a hálót felépítő osztályok és metódusok pedig a model.ipynb-ből származnak
Kezdjük a modulok hivatkozásával

In [ ]:
import sys
sys.path.append('c:\\users\\ifjto\\appdata\\local\\programs\\python\\python37\\lib\\site-packages')
import numpy as np
import import_ipynb
from model import *
import numpy as np
import torch
from torch import nn
from torch.utils.data import *
import torch.optim as optim
import torch.cuda
import torchvision.transforms as transforms
import cv2
import os
import time
from IPython.display import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

Nézzük meg, hogy tudunk-e videókártyán futtatni.

In [ ]:
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("CUDA available!")
else:
    print("CUDA not available!")

Konstansok.

In [ ]:
# the maximum number of pictures included in each epoch
TRAIN_SIZE = 8000
AUTO_LR = pow(10, -3.651289612696951)
DISC_LR = pow(10, -6.587095973689431)
NUM_EPOCH = 30
WD = pow(10, -4.2643240292910685)
nFeat = 118
SWD = 4
dropout = 0.46873611570554313
RATIO = 0.41148778160552374

Konkrét adatok beolvasása. Kötegméretnek célszerű 2 hatványt használni, a gyorsaság növelése érdekében.

In [ ]:
trainloader, testloader, validationloader, discriminator_dataset, disc_loader = get_data(32, TRAIN_SIZE)

Konkrét autoencoder háló elkészítése. Loss függvénynek használjunk "Mean Squared Error" függvényt.

In [ ]:
autoencoder = Autoencoder(nFeat)
if torch.cuda.is_available():
    autoencoder = autoencoder.cuda()

Konkrét diszkriminátor elkészítése.

In [ ]:
discriminator = Discriminator(dropout)
if torch.cuda.is_available():
    discriminator = discriminator.cuda()

Tanítás és loss kirajzolása

In [ ]:
train_loss, disc_loss = train(
    autoencoder, discriminator, trainloader, discriminator_dataset, disc_loader, NUM_EPOCH, ratio=RATIO,
    steps_without_disc=SWD, TRAIN_SIZE=TRAIN_SIZE, AUTO_LR=AUTO_LR, DISC_LR=DISC_LR, wd=WD)
plt.figure()
plt.plot(train_loss)
plt.title('Train and Discriminator Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(disc_loss)

Állítsunk vissza egy képet próbaként!

In [ ]:
for data in testloader:
    pic, real = data
    generated = redraw(autoencoder, pic)
    print_tensor(generated, 128)
    print_tensor(pic, 32)
    print_tensor(real)
    break

In [ ]:
print(get_result(autoencoder, validationloader))